В этой задаче вам предстоит по данным из магазинов предсказать выручку.


Задание:

* Выберите несколько магазинов для анализа (нужно строить отдельно по каждому, т.к. они находятся в разных регионах)
* Постройте графики (time plot, seasonality plot, series decomposition). Что вы можете по ним сказать?
* Проведите анализ данных, посчитайте количественные характеристики датасета
* Постройте наивные предсказания, по последнему, среднему значению на последние 4 недели данных
* Проверьте точность метриками MAPE, RMSE
* Сделайте правильную кросс-валидацию
* Используйте модель SARIMAX

Опционально:

* Используйте библиотеку Prophet для предсказания (https://facebook.github.io/prophet/)
* Используйте дополнительные предикторы для предсказания
* А можно ли использовать тут XGBoost?

# Загрузка всего необходимого

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

from sklearn.preprocessing import LabelEncoder
from datetime import datetime as dt
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Lesson_13/train.csv")
df

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [ ]:
gc.enable()

# Анализ данных

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [ ]:
df.describe(include = 'all')

,Store,Dept,Date,Weekly_Sales,IsHoliday
count,421570.000000,421570.000000,421570,421570.000000,421570
unique,NaN,NaN,143,NaN,2
top,NaN,NaN,2011-12-23,NaN,False
freq,NaN,NaN,3027,NaN,391909
mean,22.200546,44.260317,NaN,15981.258123,NaN
std,12.785297,30.492054,NaN,22711.183519,NaN
min,1.000000,1.000000,NaN,-4988.940000,NaN
25%,11.000000,18.000000,NaN,2079.650000,NaN
50%,22.000000,37.000000,NaN,7612.030000,NaN
75%,33.000000,74.000000,NaN,20205.852500,NaN


In [ ]:
s = df.isna().sum()
nan = pd.DataFrame(data = s[s != 0].sort_values(ascending=False), columns=['Total NaN count'])
nan['Percent'] = df.isna().mean() * 100.0

nan.T

,Store,Dept,Date,Weekly_Sales,IsHoliday
Total NaN count,NaN,NaN,NaN,NaN,NaN
Percent,0,0,0,0,0


# Работа с датасетом

Пусть у нас нашим индексом будет дата. А isHoliday преобразуем в int. Тогда саму дату мы удаляем из значений.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.index = df['Date'].values
df.drop(columns = ['Date'], inplace=True)
df['IsHoliday'] = LabelEncoder().fit_transform(df['IsHoliday'])
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 421570 entries, 2010-02-05 to 2012-10-26
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Weekly_Sales  421570 non-null  float64
 3   IsHoliday     421570 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 16.1 MB


Неплохо... Теперь посмотрим как выглядит датасет

In [ ]:
df

,Store,Dept,Weekly_Sales,IsHoliday
2010-02-05,1,1,24924.50,0
2010-02-12,1,1,46039.49,1
2010-02-19,1,1,41595.55,0
2010-02-26,1,1,19403.54,0
2010-03-05,1,1,21827.90,0
...,...,...,...,...
2012-09-28,45,98,508.37,0
2012-10-05,45,98,628.10,0
2012-10-12,45,98,1061.02,0
2012-10-19,45,98,760.01,0


Отлично. Теперь сгрупируем по магазинам

In [ ]:
df.groupby([df.index, df.Store, df.Dept]).aggregate(np.sum)

Weekly_Sales  IsHoliday
           Store Dept                         
2010-02-05 1     1         24924.50          0
                 2         50605.27          0
                 3         13740.12          0
                 4         39954.04          0
                 5         32229.38          0
...                             ...        ...
2012-10-26 45    93         2487.80          0
                 94         5203.31          0
                 95        56017.47          0
                 97         6817.48          0
                 98         1076.80          0

[421570 rows x 2 columns]

In [ ]:
def make_analys(Store_num):
  df_new = df.loc[(df['Store'] == 8)].drop(['Store', 'Dept', 'IsHoliday'], axis=1)
  

In [ ]:
df.loc[(df['Store'] == 8)]

,Store,Dept,Weekly_Sales,IsHoliday
2010-02-05,8,1,16181.89,0
2010-02-12,8,1,34262.09,1
2010-02-19,8,1,22319.25,0
2010-02-26,8,1,11722.71,0
2010-03-05,8,1,12979.74,0
...,...,...,...,...
2012-09-14,8,99,0.04,0
2012-10-05,8,99,470.00,0
2012-10-12,8,99,50.02,0
2012-10-19,8,99,0.03,0


In [ ]:
df_new = df.loc[(df['Store'] == 8)]
df_new.sort_index(inplace=True)
decompose = seasonal_decompose(df_new['Weekly_Sales'],model='additive')
decompose.plot()
plt.show()

ValueError: ignored